In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [20]:
from otgnn.models import GCN, compute_ot
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.mol_opt import MolOpt

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time

In [3]:
data_loader = get_loader("iclr19-graph2graph/data/qed", "val", 48, True)

In [4]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [5]:
sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"
molopt = MolOpt(args).to(device = args.device)
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=50, out_features=50, bias=True)
  (opt1): Linear(in_features=50, out_features=50, bias=True)
)

In [32]:
molopt.ref.shape

torch.Size([40, 50])

In [40]:
molopt.Nref = molopt.ref.shape[0]

In [50]:
tg_embedding = torch.empty(molopt.Nref * len(X.scope), 50, device = args.device)

In [51]:
tg_embedding.shape

torch.Size([1920, 50])

In [52]:
embedding = molopt.encode(X)

In [53]:
for i, (stx, lex) in enumerate(X.scope):
    narrow = embedding.narrow(0, stx, lex)
    H = np.ones(lex)/lex
    _,_,OT_xy,_ = compute_ot(narrow, molopt.ref, H, molopt.Href, sinkhorn_entropy = 0.1, device = args.device)
    V = torch.matmul((molopt.Nref * OT_xy).T, narrow) - molopt.ref
    tg_embedding[i*molopt.Nref : (i+1)*molopt.Nref,:] = V / np.sqrt(molopt.Nref)

In [54]:
V = torch.matmul((molopt.Nref * OT_xy).T, narrow) - molopt.ref

In [56]:
tg_embedding

tensor([[-0.4105,  0.0366, -0.0232,  ..., -0.0333, -0.0045,  0.1041],
        [ 0.0519,  0.0820,  0.0207,  ..., -0.0523,  0.0890, -0.1189],
        [ 0.0362, -0.0883, -0.1759,  ...,  0.1477,  0.0641, -0.0533],
        ...,
        [ 0.1030,  0.1289, -0.1032,  ...,  0.3972, -0.0809,  0.0090],
        [ 0.1549, -0.2954, -0.2081,  ...,  0.1850, -0.0116, -0.2372],
        [ 0.1086, -0.1630,  0.0699,  ...,  0.0592, -0.0019, -0.2036]],
       device='cuda:0', grad_fn=<CopySlices>)

In [57]:
V

tensor([[-2.5960,  0.2316, -0.1467,  ..., -0.2108, -0.0283,  0.6581],
        [ 0.3284,  0.5186,  0.1309,  ..., -0.3307,  0.5630, -0.7518],
        [ 0.2288, -0.5586, -1.1126,  ...,  0.9344,  0.4057, -0.3371],
        ...,
        [ 0.6513,  0.8152, -0.6527,  ...,  2.5122, -0.5120,  0.0570],
        [ 0.9800, -1.8682, -1.3164,  ...,  1.1700, -0.0736, -1.4999],
        [ 0.6867, -1.0309,  0.4420,  ...,  0.3741, -0.0121, -1.2877]],
       device='cuda:0', grad_fn=<SubBackward0>)